In [ ]:
# ----------------------------------------
# Constantes para el juego Hundir la flota
# ----------------------------------------

import numpy as np

# Dimensión del tablero
dim_tablero = 10

# definimos los barcos
barcos = {
    "barco1": (1, 4), # siendo el primer número la eslora y el segundo número la cantidad de barcos
    "barco2": (2, 3),
    "barco3": (3, 2),
    "barco4": (4, 1)
}

# definimos los símbolos para el tablero
agua = "~"
barco = "B"
impacto = "X"
fallo = "O"
desconocido = "." # para el tablero del enemigo donde aun no se ha disparado

# damos una semilla para la generación aleatoria
np.random.seed(42)


In [ ]:
# --------------------------------------------
#  Clase Tablero para el juego Hundir la flota
# --------------------------------------------


import numpy as np
import variables

class Tablero:
    def __init__(self, id_jugador: str):
        self.id_jugador = id_jugador
        self.dimension = dim_tablero
        self.tablero_barcos = np.full((dim_tablero, dim_tablero), agua) # con los barcos propios
        self.tablero_disparos = np.full((dim_tablero, dim_tablero), desconocido) # con los disparos hechos al enemigo
        self.vidas = 0  # total de casillas con barco


In [ ]:
# 1. Función para generar las coordenadas de los barcos,
# dado una lista de barcos que contenga dos parámetros, 
# el primero la eslora, y el segundo la cantidad de barcos iguales.
# Se tiene en cuenta que no se salga del tablero y que no pise otro barco
def colocar_barcos_aleatorio(barcos, dim_tablero):

    lista_coord_ocupadas = []

    for (eslora, cantidad) in barcos.values():
        for i in range(cantidad):
            x = np.random.randint(0, dim_tablero)
            y = np.random.randint(0, dim_tablero-eslora)
            
            while [x, y] in lista_coord_ocupadas:
                x = np.random.randint(0, dim_tablero)
                y = np.random.randint(0, dim_tablero-eslora)

            lista_coord_ocupadas.append([x, y])

            for j in range(1,eslora):
                yj = y+j
                lista_coord_ocupadas.append([x, yj])

            print("eslora", eslora, "cantidad", cantidad, lista_coord_ocupadas)



eslora 1 cantidad 4 [[9, 3]]
eslora 1 cantidad 4 [[9, 3], [7, 6]]
eslora 1 cantidad 4 [[9, 3], [7, 6], [8, 7]]
eslora 1 cantidad 4 [[9, 3], [7, 6], [8, 7], [4, 1]]
eslora 2 cantidad 3 [[9, 3], [7, 6], [8, 7], [4, 1], [4, 7], [4, 8]]
eslora 2 cantidad 3 [[9, 3], [7, 6], [8, 7], [4, 1], [4, 7], [4, 8], [9, 0], [9, 1]]
eslora 2 cantidad 3 [[9, 3], [7, 6], [8, 7], [4, 1], [4, 7], [4, 8], [9, 0], [9, 1], [8, 4], [8, 5]]
eslora 3 cantidad 2 [[9, 3], [7, 6], [8, 7], [4, 1], [4, 7], [4, 8], [9, 0], [9, 1], [8, 4], [8, 5], [0, 0], [0, 1], [0, 2]]
eslora 3 cantidad 2 [[9, 3], [7, 6], [8, 7], [4, 1], [4, 7], [4, 8], [9, 0], [9, 1], [8, 4], [8, 5], [0, 0], [0, 1], [0, 2], [6, 0], [6, 1], [6, 2]]
eslora 4 cantidad 1 [[9, 3], [7, 6], [8, 7], [4, 1], [4, 7], [4, 8], [9, 0], [9, 1], [8, 4], [8, 5], [0, 0], [0, 1], [0, 2], [6, 0], [6, 1], [6, 2], [7, 0], [7, 1], [7, 2], [7, 3]]


In [ ]:
# visualizo el tablero vacío

jugador = Tablero("Lucía")
print(jugador.tablero_barcos)

[['~' '~' '~' '~' '~' '~' '~' '~' '~' '~']
 ['~' '~' '~' '~' '~' '~' '~' '~' '~' '~']
 ['~' '~' '~' '~' '~' '~' '~' '~' '~' '~']
 ['~' '~' '~' '~' '~' '~' '~' '~' '~' '~']
 ['~' '~' '~' '~' '~' '~' '~' '~' '~' '~']
 ['~' '~' '~' '~' '~' '~' '~' '~' '~' '~']
 ['~' '~' '~' '~' '~' '~' '~' '~' '~' '~']
 ['~' '~' '~' '~' '~' '~' '~' '~' '~' '~']
 ['~' '~' '~' '~' '~' '~' '~' '~' '~' '~']
 ['~' '~' '~' '~' '~' '~' '~' '~' '~' '~']]


In [ ]:
# coloco los barcos en el tablero y lo visualizo

for [x, y] in lista_coord_ocupadas:
    print(x, y)
    jugador.tablero_barcos[x, y] = "B"
print(jugador.tablero_barcos)

9 3
7 6
8 7
4 1
4 7
4 8
9 0
9 1
8 4
8 5
0 0
0 1
0 2
6 0
6 1
6 2
7 0
7 1
7 2
7 3
[['B' 'B' 'B' '~' '~' '~' '~' '~' '~' '~']
 ['~' '~' '~' '~' '~' '~' '~' '~' '~' '~']
 ['~' '~' '~' '~' '~' '~' '~' '~' '~' '~']
 ['~' '~' '~' '~' '~' '~' '~' '~' '~' '~']
 ['~' 'B' '~' '~' '~' '~' '~' 'B' 'B' '~']
 ['~' '~' '~' '~' '~' '~' '~' '~' '~' '~']
 ['B' 'B' 'B' '~' '~' '~' '~' '~' '~' '~']
 ['B' 'B' 'B' 'B' '~' '~' 'B' '~' '~' '~']
 ['~' '~' '~' '~' 'B' 'B' '~' 'B' '~' '~']
 ['B' 'B' '~' 'B' '~' '~' '~' '~' '~' '~']]
